# PyTorch를 사용한 MNIST 학습

이 예제는 MNIST Training using PyTorch의 번역본입니다.

원문 링크:
https://github.com/awslabs/amazon-sagemaker-examples/blob/master/sagemaker-python-sdk/pytorch_mnist/pytorch_mnist.ipynb

## 내용

1. [배경](#배경)
1. [셋업](#셋업)
1. [데이터](#데이터)
1. [학습](#학습)
1. [호스트](#호스트)

---

## 배경

MNIST는 필기 숫자 분류에 널리 사용되는 데이터 세트입니다. 손으로 쓴 숫자의 70,000 개 레이블이 붙은 28x28 픽셀 grayscale 이미지로 구성됩니다. 데이터 세트는 60,000 개의 훈련 이미지와 10,000 개의 테스트 이미지로 나뉩니다. 10 개의 분류가 있습니다 (10개 숫자 중 각 숫자가 1개 분류). 이 튜토리얼은 PyTorch를 사용하여 SageMaker에서 MNIST 모델을 학습하고 테스트하는 방법을 보여줍니다.

SageMaker의 PyTorch에 대한 자세한 내용은 [sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers) 및 [sagemaker-python-sdk](https://github.com/aws/sagemaker-python-sdk) github repositories를 방문하세요.

---

## 셋업

_이 노트북은 ml.m4.xlarge 노트북 인스턴스에서 작성 및 테스트되었습니다._

SageMaker 세션을 생성하고 아래처럼 지정하여 시작합니다:

- 학습 및 모델 데이터에 사용하려는 S3 버킷 및 prefix는 노트북 인스턴스, 학습 및 호스팅과 동일한 리전 내에 있어야합니다.
- 데이터에 대한 학습 및 호스팅 엑세스에 사용하는 IAM 역할 arn를 생성하는 방법은 문서를 참조하세요. 노트북 인스턴스, 학습 및 호스팅에 한 개 이상의 역할이 필요한 경우 `sagemaker.get_execution_role()`를 적절한 IAM 역할 arn 문자열로 바꿔주세요.



In [ ]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/DEMO-pytorch-mnist'

role = sagemaker.get_execution_role()

## 데이터
### 데이터 가져오기

In [ ]:
from torchvision import datasets, transforms

datasets.MNIST('data', download=True, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
]))

### S3에 데이터 업로드
`sagemaker.Session.upload_data` 함수를 사용하여 데이터 셋을 S3에 업로드 할 것입니다. inputs 변수에 데이터 위치를 식별하는 반환값을 저장하여 나중에 학습 작업을 시작할 때 사용합니다.


In [ ]:
inputs = sagemaker_session.upload_data(path='data', bucket=bucket, key_prefix=prefix)
print('input spec (in this case, just an S3 path): {}'.format(inputs))

## 학습
### 학습 스크립트
`mnist.py` 스크립트는 SageMaker 모델을 훈련시키고 호스팅하는 데 필요한 모든 코드를 제공합니다 (`model_fn` 함수로 모델 로드).
학습 스크립트는 SageMaker 외부에서 실행할 수있는 학습 스크립트와 매우 유사하지만 다음과 같은 다양한 환경 변수를 통해 학습 환경에 대한 유용한 속성에 액세스 할 수 있습니다.
예를 들면:

* `SM_MODEL_DIR`: 모델 아티팩트를 저장할 디렉토리의 경로를 나타내는 문자열입니다.
  이러한 아티팩트는 모델 호스팅을 위해 S3에 업로드됩니다.
* `SM_NUM_GPUS`: 현재 컨테이너에서 사용 가능한 gpu 갯수입니다.
* `SM_CURRENT_HOST`: 컨테이너 네트워크에서 현재 컨테이너의 이름입니다.
* `SM_HOSTS`: 모든 호스트를 포함하는 JSON 배열.

한개의 입력 채널 'training'이 PyTorch estimator의 `fit ()` 메소드 호출에서 사용되었다고 가정하면 다음은 `SM_CHANNEL_ [channel_name]` 형식으로 설정됩니다.

* `SM_CHANNEL_TRAINING`: 'training' 채널에 데이터가 포함 된 디렉토리의 경로를 나타내는 문자열.

학습 환경 변수에 대한 자세한 내용은 [SageMaker Containers](https://github.com/aws/sagemaker-containers)를 확인하세요.

일반적인 학습 스크립트는 입력 채널에서 데이터를 로드하고, 하이퍼파라미터를 사용한 교육을 구성하고, 모델을 교육하고, 나중에 호스팅 할 수 있도록 모델을`model_dir`에 저장합니다. 하이퍼파라미터는 인수로 스크립트에 전달되며 `argparse.ArgumentParser` 인스턴스를 사용하여 조회 할 수 있습니다.

SageMaker가 학습 스크립트를 import해오기 때문에 이 예제에서와 동일한 스크립트를 사용하여 모델을 호스팅하는 경우 학습 코드를 main guard (``if __name __ == '__ main __':``)에 배치하여 SageMaker가 실수로 코드를 실행하지 않도록 합니다.

예를 들어 이 노트북에서 실행되는 스크립트:

In [ ]:
!pygmentize mnist.py

### SageMaker에서 학습 실행

`PyTorch` 클래스를 통해 SageMaker 인프라에서 우리의 학습 함수를 학습 작업으로 실행할수 있게됩니다. 이를 학습 스크립트, IAM 역할, 학습 인스턴스 수, 학습 인스턴스 유형 및 하이퍼파라미터를 사용하여 구성해야합니다. 이 예제에서 우리는 2 개의 ``ml.c4.xlarge`` 인스턴스에서 학습 작업을 실행할 것입니다. 그러나 이 예제는 하나 또는 여러 개의 CPU 또는 GPU 인스턴스에서 실행할 수 있습니다 ([사용 가능한 전체 인스턴스 목록] (https://aws.amazon.com/sagemaker/pricing/instance-types/)). hyperparameters 매개 변수는 학습 스크립트에 전달 될 값의 dict입니다. 위의`mnist.py` 스크립트에서 이 값들에 액세스하는 방법을 확인할 수 있습니다.

In [ ]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point='mnist.py',
                    role=role,
                    framework_version='1.2.0',
                    train_instance_count=2,
                    train_instance_type='ml.c4.xlarge',
                    hyperparameters={
                        'epochs': 6,
                        'backend': 'gloo'
                    })

'PyTorch'객체를 생성 한 후 S3에 업로드 한 데이터를 사용하여 객체에 fit할 수 있습니다. SageMaker를 사용하면 로컬 파일 시스템에서 데이터를 사용할 수 있으므로 학습 스크립트가 디스크에서 데이터를 쉽게 읽을 수 있습니다.


In [ ]:
estimator.fit({'training': inputs})

## 호스트
### 엔드포인트 생성
학습 완료 후, 우리는 `PyTorch` estimator 객체를 사용하여 `PyTorchPredictor`를 빌드하고 배포합니다. 그러면 추론을 수행하는 데 사용할 수있는 호스팅 된 예측 서비스 인 Sagemaker Endpoint가 생성됩니다.

위에서 언급했듯이 우리는 `mnist.py` 스크립트에서 `model_fn`을 구현했습니다. [sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers)에 정의 된 `input_fn`,`predict_fn`,`output_fn` 및`transform_fm`의 기본 구현을 사용할 것입니다.

deploy 함수의 인수로 엔드 포인트에 사용될 인스턴스 수와 유형을 설정할 수 있습니다. 이것은 우리가 훈련 작업에 사용한 값과 같을 필요는 없습니다. 예를 들어 GPU 기반 인스턴스 집합에서 모델을 학습 한 다음 엔드 포인트를 여러 CPU 기반 인스턴스에 배포 할 수 있지만 모델을 CPU 모델로 반환 또는 저장해야합니다. 이 것은 `mnist.py`에서 우리가 한 일과 비슷합니다. 이 예제에서는 한개 ``ml.m4.xlarge`` 인스턴스에 모델을 배포합니다.

In [ ]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

### 평가
이제 이 predictor를 사용하여 손으로 쓴 숫자를 분류 할 수 있습니다. 이미지 박스에 그림을 그리면 픽셀 데이터가 이 노트북의 `data` 변수에 로드되고, 이 변수를 `predictor`에 전달할 수 있습니다.

In [ ]:
from IPython.display import HTML
HTML(open("input.html").read())

In [ ]:
import numpy as np

image = np.array([data], dtype=np.float32)
response = predictor.predict(image)
prediction = response.argmax(axis=1)[0]
print(prediction)

### 정리

이 예제를 완료 한 후 예측 엔드 포인트를 삭제하여 연관된 인스턴스를 해제하십시오.

In [ ]:
estimator.delete_endpoint()